## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-13-20-45-42 +0000,bbc,Trump defends US Attorney General Pam Bondi ov...,https://www.bbc.com/news/articles/c9qxd2exqgno
1,2025-07-13-20-32-40 +0000,nyt,"Macron Increases Military Spending, Saying Fre...",https://www.nytimes.com/2025/07/13/world/europ...
2,2025-07-13-20-31-43 +0000,nypost,Nvidia CEO to hold media briefing in Beijing t...,https://nypost.com/2025/07/13/business/nvidia-...
3,2025-07-13-20-28-06 +0000,nyt,Children Among at Least 10 Killed in Israeli S...,https://www.nytimes.com/2025/07/13/world/middl...
4,2025-07-13-20-04-26 +0000,bbc,RSF storms cattle market and prison in 'death ...,https://www.bbc.com/news/articles/cn7d84k3r43o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,8
27,killed,5
42,tariff,3
115,new,3
64,was,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
32,2025-07-13-00-39-10 +0000,nypost,It’s no surprise the Trump DOJ confirmed Jeffr...,https://nypost.com/2025/07/12/business/its-no-...,29
31,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...,23
20,2025-07-13-15-18-43 +0000,nyt,"To Sidestep Trump Tariffs, Asian Nations Seek ...",https://www.nytimes.com/2025/07/13/business/ec...,21
15,2025-07-13-16-29-05 +0000,bbc,Gaza officials say children killed in strike a...,https://www.bbc.com/news/articles/c0rvxjnvv71o,19
30,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...,19


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
32,29,2025-07-13-00-39-10 +0000,nypost,It’s no surprise the Trump DOJ confirmed Jeffr...,https://nypost.com/2025/07/12/business/its-no-...
31,23,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
41,19,2025-07-12-21-11-05 +0000,bbc,The mushroom killer was obsessed with true cri...,https://www.bbc.com/news/articles/c0m8glx2zleo
26,17,2025-07-13-07-53-21 +0000,nypost,New Jersey tops nation’s highest property tax ...,https://nypost.com/2025/07/13/real-estate/new-...
15,14,2025-07-13-16-29-05 +0000,bbc,Gaza officials say children killed in strike a...,https://www.bbc.com/news/articles/c0rvxjnvv71o
19,14,2025-07-13-15-20-08 +0000,bbc,Texas camp buildings were removed from map sho...,https://www.bbc.com/news/articles/ce8zjk5yx8wo
35,14,2025-07-12-23-45-33 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
36,12,2025-07-12-23-40-23 +0000,bbc,Inside the Congolese mine vital to mobile phon...,https://www.bbc.com/news/articles/cyvj986l615o
13,12,2025-07-13-16-38-09 +0000,bbc,EU delays retaliatory trade tariffs against US,https://www.bbc.com/news/articles/cdez5w61g5lo
30,11,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
